# Entrega 1 - Introducción al Procesamiento del Lenguaje Natural 2018 


Verifique que su entorno de Python 3 contiene todas las bibliotecas necesarias. Importe las bibliotecas nltk y sklearn y verifique que se importan sin errores.

In [11]:
import nltk
import sklearn
import json
import os
from random import shuffle, seed

## Lectura de datos

Observe el corpus contenido en el directorio *restaurante-review-dataset* extraído de [1]. Cargue el contenido del corpus en una variable de nombre *corpus*. Utilice como estructura de datos una lista de pares (comentario, valor), donde comentario es una string con el comentario y valor corresponde a la string "POS" o "NEG" si el comentario es positivo o negativo, respectivamente. Despliegue la cantidad de comentarios positivos y negativos.
 
[1] Dubiau, L., & Ale, J. M. (2013). Análisis de Sentimientos sobre un Corpus en Español: Experimentación con un Caso de Estudio. In Proceedings of the 14th Argentine Symposium on Artificial Intelligence, ASAI (pp. 36-47).


In [18]:
path = "../dataset/restaurante-review-dataset/"
posPath = path + "pos/"
negPath = path + "neg/"
posFiles = os.listdir(posPath)
negFiles = os.listdir(negPath)
posJsons = [json.load(open(posPath + file)) for file in posFiles]
negJsons = [json.load(open(negPath + file)) for file in negFiles]
posCorpus = [(comentario,"POS") for comentario in posJsons]
negCorpus = [(comentario,"NEG") for comentario in negJsons]
corpus = posCorpus + negCorpus
print(str(len(posCorpus)) + " comentarios positivos y " + str(len(negCorpus)) + " comentarios negativos.")

3786 comentarios positivos y 3636 comentarios negativos.
['Ambiente muy familiero, para toda la comunidad y para los verdolagas especialmente. Voy siempre con mi viejo cuando Ferro juega de visitante y es cábala. Al menos eso queremos que sea, para repetir el rito una y otra vez. Ampliamente recomendable.']


Guarde el conjunto de oraciones en una lista llamada *oraciones*. Mezce esta lista aleatoriamente. Utilice una semilla para el generador aleatorio para que sus resultados sean reproducibles. Despliegue las primeros 3 comentarios.

In [15]:
oraciones = corpus # ??
seed(3)
shuffle(oraciones)

## Tokenización

Transforme cada elemento de *comentarios* en una lista de palabras utilizando las funciones de tokenización de oraciones de nltk. Almacene el resultado en una variable llamada *corpus_tokens*.

Muestre los dos últimos elementos de *corpus_tokens*.

Realice otra tokenización de los *comentarios* utilizando expresiones regulares y el módulo re de Python, en esta oportunidad, cumpla las siguientes restricciones:

- Reconocer tokens de por lo menos 3 caracteres de largo.
- No reconocer símbolos de puntuación.
- No reconocer tokens únicamente numéricos.

Almacene el resultado en una variable llamada *corpus_tokens2*.

Despliegue los dos últimos elementos de *corpus_tokens2*.

# Bag of Words

Utilice la clase CountVectorizer del módulo sklearn.feature_extraction.text para crear un objeto 'transf' que transforme el corpus en un diccionario de las palabras más representativas de las oraciones del corpus.
Investigue cada uno de los parámetros que acepta su constructor y configurelos de la forma más adecuada. Justifique sus decisiones.

Entrene la transformación e imprima las palabras resultantes en el diccionario. Imprima además la cantidad de palabras obtenidas en el diccionario.

Transforme todos los *comentarios* en vectores utilizando la transformación creada. Guarde las transformaciones en una lista llamada 'vectores':

## Similitud

Escriba una función calcular_similitud(vect1, vect2) que dados dos vectores, devuelva un número en el rango [0, 1] donde 1 significa que los vectores son idénticos y 0 que son completamente diferentes. Puede utilizar medidas como la implementada en la función cosine del módulo scipy.spatial.distance u otra que considere adecuada.

Vamos a definir los siguientes dos diccionarios:

In [93]:
similar = {}
similitud = {}

El primer diccionario guardará para cada oración, el índice de la oración más similar a ella. Esto es, similar[i] = j indica que la j-esima oración, es la oración más similar a la i-esima oración.
El segundo diccionario guardará el valor de similitud correspondiente. Si similiar[i] = j, entonces similitud[i] = calcular_similitud(oracion[i], oracion[j]).
Implemente una fución llamada calcular_similitudes(vectores, similar, similitud) que dada la lista de vectores, construya los diccionarios similar y similitud:

Implemente una función llamada *imprimir_similares* que imprima los primeros N pares de comentarios más similares del corpus. Utilice un valor que considere adecuado para N.

Ejecute las funciones calcular_similitudes e imprimir_similares. Utilice una porción del corpus que considere adecuada para que ejecute en un tiempo razonable (ej. los primeros 1000 comentarios). Despliegue los resultados y el tiempo transcurrido. Si lo desea puede incluir resultados en un bloque de texto al considerar una porción mayor. De ser así incluya la cantidad de elementos considerados y el tiempo transcurrido. 

Analice los resultados obtenidos. ¿Que observa? ¿En qué opina que se podría mejorar?

## Bag of Words (con stemming)

Utilice el parámetro 'tokenizer' de la clase CountVectorizer y conjuntamente la clase SpanishStemmer del módulo nltk.stem.snowball. Repita los experimentos realizados en la parte anterior utilizando el stemmer. Realice las comparaciones que considere pertinentes.

Al igual que en la parte anterior muestre los resultados para una porción del corpus y si lo desea incluya los resultados para una porción mayor. Despliegue el tiempo transcurrido.

Justifique las decisiones tomadas y realice los comentarios que considere adecuados. Analice los resultados obtenidos. Compare los resultados con los obtenidos anteriormente. Comente los problemas que detecte en este enfoque.